# DB Connection

In [4]:
import cassandra 
print (cassandra.__version__)
import pandas
print(pandas.__version__)
import pandas as pd
import uuid

3.25.0
1.0.1


In [2]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
from cassandra.concurrent import execute_concurrent_with_args
cloud_config1 = {
        'secure_connect_bundle': '../secure-connect-capstone-project.zip'
}
auth_provider1 = PlainTextAuthProvider('JJvGOjuOPszNbfeoOwEinlHI', 'ZM-OR5KZAIMzqksmn01xobmZW4cNRUiIft0SnYuhW966f++ILhp+2Mq,AErDqKAF7Y0AF4ReSYLjTCgsLCqRJiL3+8uxJMoS-K6qMtM1vEmgi5IT5c62OvBJkhxBzq5T')
cluster1 = Cluster(cloud=cloud_config1, auth_provider=auth_provider1)
#connect to keyspace_name
session1 = cluster1.connect('iac689')

cloud_config2= {
        'secure_connect_bundle': '../secure-connect-capstone-project2.zip'
}
auth_provider2 = PlainTextAuthProvider('OZTxZCgzatSukjKBoWpZimZs', 'cfph4pZMPt-REY_B0BTCZ-qxZfwgpIeZA2T69DIzccYlZ2I1+8gKKk6Ruk+G+70.CyFNoZymWef1Eo1KJZk,SjYbo9SgRZwY97JfG6J8uFLvNEJ8,nCjMbcGz.wgmlJC')
cluster2 = Cluster(cloud=cloud_config2, auth_provider=auth_provider2)
#connect to keyspace_name
session2 = cluster2.connect('iac689')

In [3]:
query = "SELECT * FROM system_schema.keyspaces;"
df = pd.DataFrame(list(session1.execute(query)))
df

,keyspace_name,durable_writes,graph_engine,replication
0,system_auth,True,None,"[class, us-east1]"
1,system_schema,False,None,[class]
2,data_endpoint_auth,True,None,"[class, us-east1]"
3,system,False,None,[class]
4,iac689,True,None,"[class, us-east1]"
5,datastax_sla,True,None,"[class, us-east1]"
6,system_traces,True,None,"[class, us-east1]"


In [4]:
query = "SELECT * FROM system_schema.tables WHERE keyspace_name='iac689';"
df = pd.DataFrame(list(session1.execute(query)))
df

,keyspace_name,table_name,additional_write_policy,bloom_filter_fp_chance,caching,cdc,comment,compaction,compression,crc_check_chance,...,flags,gc_grace_seconds,id,max_index_interval,memtable_flush_period_in_ms,min_index_interval,nodesync,read_repair,read_repair_chance,speculative_retry
0,iac689,data_1,99PERCENTILE,0.01,"[keys, rows_per_partition]",None,,[class],"[chunk_length_in_kb, class]",1.0,...,(compound),864000,c78013f0-7f00-11ec-b88b-b18210f12434,2048,0,128,None,BLOCKING,0.0,99PERCENTILE
1,iac689,data_2,99PERCENTILE,0.01,"[keys, rows_per_partition]",None,,[class],"[chunk_length_in_kb, class]",1.0,...,(compound),864000,aeb6fca0-7f03-11ec-b88b-b18210f12434,2048,0,128,None,BLOCKING,0.0,99PERCENTILE


In [5]:
query = "SELECT * FROM system_schema.columns WHERE keyspace_name='iac689' and table_name = 'data_1';"
df = pd.DataFrame(list(session1.execute(query)))
df

,keyspace_name,table_name,column_name,clustering_order,column_name_bytes,kind,position,required_for_liveness,type
0,iac689,data_1,active,none,b'active',regular,-1,False,float
1,iac689,data_1,reading_id,none,b'reading_id',regular,-1,False,int
2,iac689,data_1,start_mileage,none,b'start_mileage',regular,-1,False,float
3,iac689,data_1,start_time,none,b'start_time',regular,-1,False,date
4,iac689,data_1,truck_id,none,b'truck_id',regular,-1,False,int
5,iac689,data_1,truck_name,none,b'truck_name',regular,-1,False,text
6,iac689,data_1,type,none,b'type',regular,-1,False,text
7,iac689,data_1,uid,none,b'uid',partition_key,0,False,uuid


# DB_Insert

In [7]:
import pandas as pd
import glob

In [8]:
path_list = glob.glob('../cap_data2/*.csv')
df_list = []
for i in path_list:
    df_list.append(pd.read_csv(i))

/Users/vathanahim/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
for i in range(len(df_list)):
    if (i < 5):
        df_list[i]['time'] = pd.to_datetime(df_list[i]['time']).dt.date
        df_list[i]['truck_name'] = df_list[i]['truck_name'].str.replace(r"[a-zA-Z|-]","")
        df_list[i]['truck_name'] = 'abc-'+df_list[i]['truck_id'].astype(str)
    else:
        df_list[i]['start_time'] = pd.to_datetime(df_list[i]['start_time']).dt.date
        df_list[i]['truck_name'] = df_list[i]['truck_name'].str.replace(r"[a-zA-Z|-]","")
        df_list[i]['truck_name'] = 'abc-'+df_list[i]['truck_id'].astype(str)
       

In [ ]:

#Insert for the last dataframe into a different table (kind of slow)
query = "INSERT INTO data_1 (uid, active, reading_id, start_mileage, start_time, truck_id, truck_name, type) VALUES (?,?,?,?,?,?,?,?)"
prepared = session1.prepare(query)
for i in df.itertuples():
    print(i.truck_id)
    session1.execute(prepared, (uuid.uuid1(), i.active, i.reading_id, i.start_mileage, i.start_time, i.truck_id, i.truck_name, i.not_active))

In [26]:
df = pd.concat(df_list[:5])
df = df.pivot_table(index=['time', 'truck_id'], columns='type',
                     values='value', aggfunc='mean').reset_index()
df['uuid'] = df.index.to_series().map(lambda x: uuid.uuid4())
df

type,time,truck_id,Distance(miles) at WheelBasedVehicleSpeed(mph),Distance(miles) at different Altitudes,Distance(miles) at different Engine Speeds (rpms),Distance(miles) at different RoadInclination(%),Time(hours) at WheelBasedVehicleSpeed(mph),Time(hours) at different Altitudes,Time(hours) at different Engine Speeds (rpms),Time(hours) at different RoadInclination(%),uuid
0,2020-12-28,286,9.186716e+04,162463.500000,8.891520e+04,6.946500e+04,1.199557e+05,1.815138,8.876727e+04,6.934943e+04,25fd3ad8-f4a5-4c31-8b30-fc47985645bc
1,2020-12-29,286,9.231500e+04,144921.500000,8.947280e+04,6.990062e+04,1.206679e+05,1.781807,8.929427e+04,6.976115e+04,1ff2b0bd-2a53-4433-8f2d-cf100fdc523a
2,2020-12-30,286,9.554131e+04,119689.500000,9.036293e+04,7.059604e+04,1.217278e+05,1.565835,9.007860e+04,7.037391e+04,dc5901d7-e46a-4441-afa2-11fc10c26c70
3,2020-12-31,286,9.957342e+04,148580.433333,9.080247e+04,7.093943e+04,1.224575e+05,1.470555,9.061859e+04,7.079578e+04,42636c99-40c5-4331-b397-d8f433b36fa3
4,2021-01-01,207,3.788795e+06,0.000000,2.803708e+06,2.190397e+06,3.788795e+06,0.001250,2.803708e+06,2.190397e+06,9ebc8347-3be6-4fc6-a74d-4aaa3a9ab604
...,...,...,...,...,...,...,...,...,...,...,...
4542,2022-01-08,294,2.007097e+06,599411.000000,1.485252e+06,NaN,2.006177e+06,0.858795,1.484571e+06,NaN,e72c641b-a610-487a-8413-51f383362802
4543,2022-01-08,296,2.000050e+07,0.000000,1.480037e+07,1.156279e+07,1.999618e+07,0.000000,1.479718e+07,1.156029e+07,d3de7ee2-f259-45f0-af0a-4f9d347cb60e
4544,2022-01-09,208,1.028107e+07,NaN,7.607992e+06,5.943743e+06,1.027035e+07,NaN,7.600056e+06,5.937544e+06,a29b4bff-b18e-4970-bdcf-3a8f809050b3
4545,2022-01-09,209,1.365595e+06,135474.000000,1.010540e+06,7.894844e+05,1.365595e+06,0.255278,1.010540e+06,7.894844e+05,ef58ddfa-1a3d-49a1-b612-a400d25e8a74


In [28]:
l = []
for i in df[['uuid','time','Distance(miles) at different Altitudes','Distance(miles) at different Engine Speeds (rpms)','Distance(miles) at different RoadInclination(%)', 'Distance(miles) at WheelBasedVehicleSpeed(mph)','Time(hours) at different Altitudes', 'Time(hours) at different Engine Speeds (rpms)','Time(hours) at different RoadInclination(%)','Time(hours) at WheelBasedVehicleSpeed(mph)','truck_id']].columns:
    l.append(df[i].tolist())



In [43]:
query = """insert into data_2 (ui , datel , distance_at_diff_altitude , distance_at_diff_engine_speeds , distance_at_diff_roadinclination , distance_at_wheelbased_vehicle_speed , time_at_diff_altitude , time_at_diff_engine_speeds , time_at_diff_roadinclination , time_at_wheelbased_vehicle_speed , truck_id)
values (?,?,?,?,?,?,?,?,?,?,?);"""
prepared = session2.prepare(query)
execute_concurrent_with_args(session2, prepared, zip(l[0],l[1],l[2],l[3],l[4],l[5],l[6],l[7],l[8],l[9],l[10]))

[ExecutionResult(success=True, result_or_exc=<cassandra.cluster.ResultSet object at 0x7f9938757e90>),
 ExecutionResult(success=True, result_or_exc=<cassandra.cluster.ResultSet object at 0x7f9938743350>),
 ExecutionResult(success=True, result_or_exc=<cassandra.cluster.ResultSet object at 0x7f993876e350>),
 ExecutionResult(success=True, result_or_exc=<cassandra.cluster.ResultSet object at 0x7f9896ac07d0>),
 ExecutionResult(success=True, result_or_exc=<cassandra.cluster.ResultSet object at 0x7f993876e550>),
 ExecutionResult(success=True, result_or_exc=<cassandra.cluster.ResultSet object at 0x7f9938760d10>),
 ExecutionResult(success=True, result_or_exc=<cassandra.cluster.ResultSet object at 0x7f98968b3350>),
 ExecutionResult(success=True, result_or_exc=<cassandra.cluster.ResultSet object at 0x7f9938743c10>),
 ExecutionResult(success=True, result_or_exc=<cassandra.cluster.ResultSet object at 0x7f9896ac0750>),
 ExecutionResult(success=True, result_or_exc=<cassandra.cluster.ResultSet object a

In [45]:
query = "select * from iac689.data_1;"
df = pd.DataFrame(list(session1.execute(query)))
df

,uid,active,reading_id,start_mileage,start_time,truck_id,truck_name,type
0,1b644c6e-7f03-11ec-99b7-acde48001122,5400.000000,103995,15906600.0,2021-02-19,205,abc-205,Distance(miles) with CC active
1,1ba064e2-7f03-11ec-99b7-acde48001122,7340.000000,103997,15912000.0,2021-02-19,205,abc-205,Distance(miles) with CC active
2,d847cb7c-7f02-11ec-99b7-acde48001122,11815.000000,103358,12433600.0,2021-01-13,205,abc-205,Distance(miles) with CC active
3,6a9d1424-7f02-11ec-99b7-acde48001122,0.021667,99698,17789500.0,2021-01-11,284,abc-284,Time(hours) with CC Active
4,f6d2f940-7f02-11ec-99b7-acde48001122,0.138056,103646,14102000.0,2021-02-01,205,abc-205,Time(hours) with CC Active
...,...,...,...,...,...,...,...,...
8525,eb7bfd62-7f02-11ec-99b7-acde48001122,0.000000,103539,13469200.0,2021-01-25,205,abc-205,Distance(miles) with CC active
8526,fe85cb18-7f02-11ec-99b7-acde48001122,0.115278,103720,14719100.0,2021-02-05,205,abc-205,Time(hours) with CC Active
8527,17da6c18-7f03-11ec-99b7-acde48001122,0.254722,103961,15560800.0,2021-02-16,205,abc-205,Time(hours) with CC Active
8528,5300048e-7f02-11ec-99b7-acde48001122,0.130556,86044,8068540.0,2021-02-15,294,abc-294,Time(hours) with CC Active


In [44]:
query = "select * from iac689.data_2;"
df = pd.DataFrame(list(session2.execute(query)))
df

,ui,datel,distance_at_diff_altitude,distance_at_diff_engine_speeds,distance_at_diff_roadinclination,distance_at_wheelbased_vehicle_speed,time_at_diff_altitude,time_at_diff_engine_speeds,time_at_diff_roadinclination,time_at_wheelbased_vehicle_speed,truck_id
0,1c9a40f7-28bd-448b-837f-7e6f26867cee,2021-01-12,2.468247e+05,7.139896e+05,5.578044e+05,9.648508e+05,0.318268,7.137108e+05,5.575866e+05,9.644741e+05,284
1,eed85ca4-db7c-42a3-b4f6-46660ac6f5ab,2021-10-01,0.000000e+00,1.087964e+07,8.499720e+06,1.470222e+07,0.000000,1.087643e+07,8.497214e+06,1.469788e+07,296
2,69b49f87-6525-4203-8168-a406da377443,2021-11-18,NaN,1.010224e+07,7.892374e+06,1.365167e+07,NaN,1.010193e+07,7.892130e+06,1.365125e+07,282
3,375516c0-521f-4411-b83a-74f84ab454be,2021-05-12,5.921890e+05,5.071094e+05,NaN,6.852830e+05,1.227917,5.051501e+05,NaN,6.826352e+05,294
4,3f15f13f-62e0-435a-910b-317c3fda47ac,2021-01-25,NaN,2.643425e+07,2.065176e+07,3.572196e+07,NaN,2.643222e+07,2.065017e+07,3.571922e+07,279
...,...,...,...,...,...,...,...,...,...,...,...
4542,2d46a503-3581-40eb-9ad9-124e7a362185,2021-07-27,5.230573e+05,8.439053e+05,NaN,1.140412e+06,0.531806,8.434997e+05,NaN,1.139864e+06,205
4543,9932ae23-5a33-42f1-9a87-ec803a18c325,2021-08-06,8.052310e+05,8.942562e+05,NaN,1.208454e+06,4.340280,8.896362e+05,NaN,1.202211e+06,294
4544,c0b4c700-96ab-438e-a272-e45ba97a3974,2021-06-03,2.373810e+06,2.268493e+05,NaN,3.065532e+05,2.066203,2.259348e+05,NaN,3.053172e+05,292
4545,e29ed179-f997-429f-b592-8158bd7b2030,2021-10-11,NaN,5.238979e+06,4.092952e+06,7.079684e+06,NaN,5.232000e+06,4.087500e+06,7.070270e+06,208
